In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

train_hh_features = pd.read_csv('/content/drive/MyDrive/Poverty_prediction_challenge/train_hh_features.csv')
train_hh_gt = pd.read_csv('/content/drive/MyDrive/Poverty_prediction_challenge/train_hh_gt.csv')
train_rates_gt = pd.read_csv('/content/drive/MyDrive/Poverty_prediction_challenge/train_rates_gt.csv')
test_hh_features = pd.read_csv('/content/drive/MyDrive/Poverty_prediction_challenge/test_hh_features.csv')

df = train_hh_features.merge(
    train_hh_gt[['hhid', 'cons_ppp17', 'survey_id']],
    on='hhid',
    how='inner',
    validate='one_to_one'
)

if 'survey_id' in df.columns:
    print("survey_id 컬럼이 성공적으로 병합되었습니다.")

print("최종 shape:", df.shape)
display(df.head())

assert (df['survey_id_x'] == df['survey_id_y']).all()

df = df.drop(columns=['survey_id_y'])
df = df.rename(columns={'survey_id_x': 'survey_id'})

print("정리 후 컬럼명:", 'survey_id' in df.columns)
print("최종 shape:", df.shape)
df.head()

Mounted at /content/drive
최종 shape: (104234, 90)


,hhid,com,weight,strata,utl_exp_ppp17,male,hsize,num_children5,num_children10,num_children18,...,consumed4400,consumed4500,consumed4600,consumed4700,consumed4800,consumed4900,consumed5000,survey_id_x,cons_ppp17,survey_id_y
0,100001,1,75,4,594.80627,Female,1,0,0,0,...,No,No,Yes,Yes,Yes,Yes,No,100000,25.258402,100000
1,100002,1,150,4,1676.27230,Female,2,0,0,0,...,No,No,No,Yes,Yes,No,No,100000,16.996706,100000
2,100003,1,375,4,506.93719,Male,5,0,0,2,...,No,Yes,Yes,Yes,Yes,No,Yes,100000,13.671848,100000
3,100004,1,375,4,824.61786,Male,5,0,0,1,...,No,No,No,Yes,Yes,No,No,100000,7.189475,100000
4,100005,1,525,4,351.47644,Male,7,1,0,0,...,No,Yes,No,Yes,Yes,Yes,No,100000,12.308855,100000


정리 후 컬럼명: True
최종 shape: (104234, 89)


,hhid,com,weight,strata,utl_exp_ppp17,male,hsize,num_children5,num_children10,num_children18,...,consumed4300,consumed4400,consumed4500,consumed4600,consumed4700,consumed4800,consumed4900,consumed5000,survey_id,cons_ppp17
0,100001,1,75,4,594.80627,Female,1,0,0,0,...,No,No,No,Yes,Yes,Yes,Yes,No,100000,25.258402
1,100002,1,150,4,1676.27230,Female,2,0,0,0,...,No,No,No,No,Yes,Yes,No,No,100000,16.996706
2,100003,1,375,4,506.93719,Male,5,0,0,2,...,Yes,No,Yes,Yes,Yes,Yes,No,Yes,100000,13.671848
3,100004,1,375,4,824.61786,Male,5,0,0,1,...,Yes,No,No,No,Yes,Yes,No,No,100000,7.189475
4,100005,1,525,4,351.47644,Male,7,1,0,0,...,No,No,Yes,No,Yes,Yes,Yes,No,100000,12.308855


In [ ]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def prepare_features(df: pd.DataFrame, state: dict = None, fit: bool = False):

    df = df.copy()
    state = {} if state is None else state

     # 1) hsize 불일치 수정(옵션 D1)
    comp_sum = (df["num_children5"] + df["num_children10"] + df["num_children18"] +
                df["num_adult_female"] + df["num_adult_male"] + df["num_elderly"])
    inconsistent = df["hsize"] < comp_sum
    df["hsize_fixed"] = df["hsize"].where(~inconsistent, comp_sum)

    #agri
    # Check if 'sector1d' column exists before processing
    if "sector1d" in df.columns:
        s = df["sector1d"]
        # 결측치 플래그
        df["is_sector_missing"] = s.isna().astype(int)
        # 문자열 정리(공백, 대소문자, NaN 안전)
        s_clean = s.fillna("").astype(str).str.strip()
        # Agriculture, hunting and forestry / Fishing
        df["is_agri"] = (s_clean == "Agriculture, hunting and forestry").astype(int)
        df["is_fish"] = (s_clean == "Fishing").astype(int)
        # 합친 플래그
        df["is_agri_fish"] = ((df["is_agri"] == 1) | (df["is_fish"] == 1)).astype(int)
    else:
        # If 'sector1d' is not present, create the columns with default values
        df["is_sector_missing"] = 1 # Assume missing if column is absent
        df["is_agri"] = 0
        df["is_fish"] = 0
        df["is_agri_fish"] = 0

    #binary
    binary_cols = [
        "male", "owner", "water", "toilet", "sewer", "elect",
        "employed", "urban", "any_nonagric"
    ]
    consumed_cols = [c for c in df.columns if c.startswith("consumed")]
    binary_cols_extended = [c for c in binary_cols if c in df.columns] + consumed_cols

    def to_binary_series(s):
        if pd.api.types.is_numeric_dtype(s):
            return s.fillna(0).astype(int)

        mapping = {
            "Yes": 1, "No": 0,
            "Male": 1, "Female": 0,
            "Access": 1, "No access": 0,
            "Employed": 1, "Not employed": 0,
            "Urban": 1, "Rural": 0,
            "Owner": 1, "Not owner": 0
        }
        return s.map(mapping).fillna(0).astype(int)

    for col in binary_cols_extended:
        df[col] = to_binary_series(df[col])

    #label encoding
    if "educ_max" in df.columns:
        educ_map = {
            "Complete Tertiary Education": 6,
            "Complete Secondary Education": 5,
            "Incomplete Tertiary Education": 4,
            "Complete Primary Education": 3,
            "Incomplete Secondary Education": 2,
            "Incomplete Primary Education": 1,
            "Never attended": 0
        }
        if not pd.api.types.is_numeric_dtype(df["educ_max"]):
            df["educ_max"] = df["educ_max"].map(educ_map).fillna(0).astype(int)
        else:
            df["educ_max"] = df["educ_max"].fillna(0).astype(int)

    #one-hot Encoding
    multi_cols = [c for c in ["water_source", "sanitation_source", "sector1d", "dweltyp"] if c in df.columns]
    for col in multi_cols:
        df[col] = df[col].fillna("Missing")

    df = pd.get_dummies(df, columns=multi_cols, prefix=multi_cols, drop_first=False)

   # 3) 수치 결측: survey_id별 중앙값
    num_cols = ["utl_exp_ppp17", "share_secondary"]
    num_cols = [c for c in num_cols if c in df.columns]

    if fit:
        med = df.groupby("survey_id")[num_cols].median()
        state["num_medians_by_survey"] = med.to_dict()

    med_dict = state["num_medians_by_survey"]
    for c in num_cols:
        df[c] = df[c].fillna(df["survey_id"].map(med_dict[c]))


    # 6) Asset PCA (PC1)
    #    - train: asset 컬럼 선정 + scaler/pca fit 저장
    #    - test : train에서 선정한 컬럼 + scaler/pca transform
    base_asset_cols = [c for c in ['water', 'toilet', 'sewer', 'elect', 'urban', 'owner'] if c in df.columns]
    asset_dummy_cols = [c for c in df.columns
                        if c.startswith('dweltyp_') or c.startswith('water_source_') or c.startswith('sanitation_source_')]
    asset_cols_all = base_asset_cols + asset_dummy_cols

    if fit:
        X_asset = df[asset_cols_all].fillna(0)
        var = X_asset.var()
        asset_cols_used = list(var[var > 0].index)

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_asset[asset_cols_used])

        pca = PCA(n_components=1)
        pc1 = pca.fit_transform(X_scaled).reshape(-1)

        df["asset_wealth_index"] = pc1

        state["asset_cols_used"] = asset_cols_used
        state["asset_scaler"] = scaler
        state["asset_pca"] = pca
        state["asset_pca_var_ratio"] = float(pca.explained_variance_ratio_[0])
    else:
        asset_cols_used = state.get("asset_cols_used", [])
        scaler = state.get("asset_scaler", None)
        pca = state.get("asset_pca", None)

        X_asset = df.reindex(columns=asset_cols_used, fill_value=0).fillna(0)
        X_scaled = scaler.transform(X_asset)
        pc1 = pca.transform(X_scaled).reshape(-1)

        df["asset_wealth_index"] = pc1

    #EDR
    adult_total = (df["num_adult_female"].fillna(0) + df["num_adult_male"].fillna(0))
    workers = df["sworkershh"].fillna(0) * adult_total

    #demo
    children_cols = ["num_children5", "num_children10", "num_children18"]
    df["total_children"] = df[children_cols].fillna(0).sum(axis=1)

    adults_cols = ["num_adult_female", "num_adult_male"]
    df["total_adults"] = df[adults_cols].fillna(0).sum(axis=1)

    # [NEW] 아동 부양비 (성인 1명당 감당해야 할 아이 수)
    # 성인이 0명인 경우 대비하여 max(1) 처리
    df["refined_dependency_ratio"] = df["total_children"] / df["total_adults"].replace(0, 1)

    # [NEW] 가구 내 아동 점유율 (hsize 대비)
    df["child_share"] = df["total_children"] / df["hsize_fixed"].replace(0, 1)

    df["no_worker"] = (workers == 0).astype(int)
    df["edr_score"] = np.where(workers > 0, (df["hsize_fixed"].fillna(0) - workers) / workers, np.nan)


    if 'sfworkershh' in df.columns:
        df['sfworkershh'] = df['sfworkershh'].fillna(0)
    else:
        df['sfworkershh'] = 0

    bins = [-np.inf, 0, 0.2, 0.4, 0.6, 0.8, 1.0]
    labels = [0, 1, 2, 3, 4, 5]

    df['sfworkershh_clip'] = df['sfworkershh'].clip(lower=0, upper=1)
    # pd.cut을 사용하여 비율을 점수로 변환
    df['sfworker_rank'] = pd.cut(df['sfworkershh_clip'], bins=bins, labels=labels, include_lowest=True)

    # 카테고리 타입을 정수형(int)으로 변환 (모델 학습용)
    df['sfworker_rank'] = df['sfworker_rank'].astype(int)

    # 하나 더 추가 변수: 정규직 인원 수 (Quantity)
    # 비율이 낮아도 가족 수가 많으면 중요하므로 이 정보는 살려둠
    df['num_regular_workers'] = df['sfworkershh'] * df['hsize']

    #여성가장
    df['is_female_head'] = df['male'].apply(lambda x: 1 if x == 0 else 0)

    #utl_exp_ppp17 log화
    df["log_utl_exp_ppp17"] = np.log(df["utl_exp_ppp17"].fillna(0) + 1)

    # Define q_low and q_high locally for this clipping logic
    local_q_low, local_q_high = 0.01, 0.99 # Use consistent values, e.g., 1st and 99th percentile

    if fit:
      caps = df.groupby("survey_id")["log_utl_exp_ppp17"].quantile([local_q_low, local_q_high]).unstack()
      # caps: index=survey_id, columns=[local_q_low, local_q_high]
      state["utl_log_caps_by_survey"] = caps.to_dict(orient="index")
      state["utl_log_cap_q_low"] = local_q_low
      state["utl_log_cap_q_high"] = local_q_high
      state["utl_log_cap_group"] = "survey_id"


    # 3) transform: 저장된 컷으로 clip
    if state.get("utl_log_cap_group") == "survey_id":
        caps = state["utl_log_caps_by_survey"]
        # Retrieve q_low/q_high used during fitting from state
        q_low_fitted = state["utl_log_cap_q_low"]
        q_high_fitted = state["utl_log_cap_q_high"]

        def clip_row(r):
            sid = r["survey_id"]
            if sid in caps:
                lo = caps[sid].get(q_low_fitted, None)
                hi = caps[sid].get(q_high_fitted, None)
                # q_low/q_high가 float key로 저장될 때 미세 오차가 걱정되면 아래 fallback 사용
                if lo is None or hi is None:
                    # 키가 문자열로 저장된 경우 대비
                    lo = caps[sid].get(str(q_low_fitted), lo)
                    hi = caps[sid].get(str(q_high_fitted), hi)
                if lo is not None and hi is not None:
                    return np.clip(r["log_utl_exp_ppp17"], lo, hi) # Changed 'utl_log'
            return r["log_utl_exp_ppp17"] # Changed 'utl_log'

        df["utl_log_qcap"] = df.apply(clip_row, axis=1)

    else:
        caps_g = state["utl_log_caps_global"]
        df["utl_log_qcap"] = df["log_utl_exp_ppp17"].clip(lower=caps_g["lo"], upper=caps_g["hi"]) # Changed 'utl_log'

    #weight
    if fit:
        wcap = df.groupby("survey_id")["weight"].quantile(0.999).to_dict()
        state["weight_cap_999_by_survey"] = wcap

    wcap = state["weight_cap_999_by_survey"]
    df["weight_capped"] = df.apply(lambda r: min(r["weight"], wcap.get(r["survey_id"], r["weight"])), axis=1)

    # Fill NaNs in object columns with a string placeholder for CatBoost compatibility
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype(str).replace('nan', 'Missing_Category')

    # 7) 최종 feature matrix 생성
    #    - train: train_cols 저장
    #    - test : train_cols에 reindex
    drop_cols = []
    for c in ["cons_ppp17", "log_cons", "survey_id", "hhid", "sworkershh", "utl_exp_ppp17", "num_children5", "num_children10", "num_children18","num_adult_female", "num_adult_male" ]:
        if c in df.columns:
            drop_cols.append(c)

    X = df.drop(columns=drop_cols)

    if fit:
        state["train_cols"] = list(X.columns)
    else:
        X = X.reindex(columns=state["train_cols"], fill_value=0)

    return X, state

In [ ]:
import numpy as np
import pandas as pd
import re
from scipy.stats import norm
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GroupKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor


# ============================================================
# 1) 기초 설정 및 Helper 함수 (원형 유지 + 필요한 최소 보강)
# ============================================================

DEFAULT_THRESHOLDS = np.array([
    3.17, 3.94, 4.60, 5.26, 5.88, 6.47, 7.06, 7.70, 8.40,
    9.13, 9.87, 10.70, 11.62, 12.69, 14.03, 15.64, 17.76, 20.99, 27.37
], dtype=float)

def infer_thresholds_from_train_rates(train_rates_gt: pd.DataFrame):
    survey_col = None
    for c in train_rates_gt.columns:
        if c.lower() in ["survey_id", "survey", "s_id"]:
            survey_col = c
            break
    if survey_col is None:
        survey_col = train_rates_gt.columns[0]

    rate_cols = [c for c in train_rates_gt.columns if c != survey_col]
    thr_pairs = []
    for c in rate_cols:
        m = re.search(r"(\d+(\.\d+)?)", str(c))
        if m:
            thr_pairs.append((float(m.group(1)), c))

    if len(thr_pairs) >= 10:
        thr_pairs.sort(key=lambda x: x[0])
        thresholds = np.array([t for t, _ in thr_pairs], dtype=float)
        ordered_cols = [col for _, col in thr_pairs]
    else:
        thresholds = DEFAULT_THRESHOLDS
        ordered_cols = rate_cols

    return thresholds, ordered_cols, survey_col

def eval_rate_metrics(rate_pred_df, train_rates_gt, survey_col="survey_id"):
    rate_cols = [c for c in train_rates_gt.columns if c != survey_col]

    gt = train_rates_gt.copy()
    gt[survey_col] = gt[survey_col].astype(str)

    rp = rate_pred_df.copy()
    rp[survey_col] = rp[survey_col].astype(str)

    merged = gt.merge(rp, on=survey_col, how="inner", suffixes=('_gt', '_pred'))

    gt_mat = merged[[c + "_gt" for c in rate_cols]].to_numpy(dtype=float)
    pr_mat = merged[[c + "_pred" for c in rate_cols]].to_numpy(dtype=float)

    diff = pr_mat - gt_mat
    rmse = float(np.sqrt(np.mean(diff**2)))
    r2 = float(r2_score(gt_mat.reshape(-1), pr_mat.reshape(-1)))
    mse = float(np.mean(diff**2))

    return rmse, mse, r2, rmse

# ============================================================
# (NEW) Survey-level sigma estimation (Method A)
# ============================================================

def make_survey_features_from_logmu(hh_df, mu_col, survey_col):
    g = hh_df.groupby(survey_col)
    out = pd.DataFrame({
        survey_col: g.size().index,
        "n_households": g.size().values,
        "mu_mean": g[mu_col].mean().values,
        "mu_std": g[mu_col].std().fillna(0).values,
        "mu_p90": g[mu_col].quantile(0.9).values,
        "mu_p10": g[mu_col].quantile(0.1).values,
    })
    return out


def compute_survey_sigma_gt(
    hh_df, mu_col, true_log_col, survey_col, weight_col
):
    tmp = hh_df[[survey_col, weight_col, mu_col, true_log_col]].copy()
    tmp["resid2"] = (tmp[true_log_col] - tmp[mu_col]) ** 2

    sigma_gt = (
        tmp.groupby(survey_col)
        .apply(lambda g: np.sqrt(np.average(g.resid2, weights=g[weight_col])))
        .reset_index(name="sigma_gt")
    )
    return sigma_gt


# ============================================================
# 2) CDF 기반 rate 계산 (중요: log-space 버전)
#    이유: stacking이 log(cons)를 예측하므로 CDF도 log-space로 맞춰야 정합적임
#    P(cons < z) = Phi((log(z) - mu_log)/sigma_log)
# ============================================================

def compute_survey_rates_CDF_log_sigma(
    hh_df, mu_log_col, sigma_col,
    survey_col, weight_col,
    thresholds, monotone_fix=True
):
    tmp = hh_df[[survey_col, weight_col, mu_log_col, sigma_col]].copy()
    tmp[survey_col] = tmp[survey_col].astype(str)

    thresholds = np.array(thresholds, dtype=float)
    logz = np.log(np.maximum(thresholds, 1e-12))
    rate_cols = [f"pct_hh_below_{t:.2f}" for t in thresholds]

    rows = []
    for sid, g in tmp.groupby(survey_col):
        w = g[weight_col].to_numpy(float)
        mu = g[mu_log_col].to_numpy(float)
        sigma = float(g[sigma_col].iloc[0])

        z = (logz[None, :] - mu[:, None]) / (sigma + 1e-12)
        probs = norm.cdf(z)

        rates = np.average(probs, weights=w, axis=0)

        row = {survey_col: int(sid) if sid.isdigit() else sid}
        for i, c in enumerate(rate_cols):
            row[c] = float(rates[i])

        if monotone_fix:
            arr = np.array([row[c] for c in rate_cols])
            ir = IsotonicRegression(increasing=True, out_of_bounds="clip")
            fixed = ir.fit_transform(np.arange(len(arr)), arr)
            for i, c in enumerate(rate_cols):
                row[c] = float(np.clip(fixed[i], 0, 1))

        rows.append(row)

    return pd.DataFrame(rows)[[survey_col] + rate_cols]


# ============================================================
# 3) Stacking 모델 (OOF + Test 예측 생성)  <= 네가 요청한 "#이부분" 대체
#    - prepare_features(df, state, fit)은 네가 이미 만든 전처리 함수 사용
#    - GroupKFold by survey_id
#    - base: LGB/XGB/CAT, meta: ElasticNet
#    - fold별로 "train에서만" 전처리 fit -> val/test transform (누수 방지)
#    - meta도 fold별 train에서만 fit -> val/test 적용 (진짜 OOF)
# ============================================================

def clean_cols_unique(cols):
    cleaned = [re.sub(r"[^A-Za-z0-9_]+", "_", str(c)) for c in cols]
    seen = {}
    out = []
    for c in cleaned:
        if c not in seen:
            seen[c] = 0
            out.append(c)
        else:
            seen[c] += 1
            out.append(f"{c}__dup{seen[c]}")
    return out

def get_fold_logmu_preds_stacking(
    train_full_df: pd.DataFrame,
    test_full_df: pd.DataFrame,
    n_splits: int = 3,
    seed: int = 42,
    q_low: float = 0.01,
    q_high: float = 0.99,
    lgb_params: dict = None,
    xgb_params: dict = None,
    cat_params: dict = None,
    lgb_early_stopping: int = 300,
    xgb_early_stopping: int = 300,
    cat_od_wait: int = 300,
    meta_cv: int = 5,
    positive_meta: bool = True
):
    tr = train_full_df.copy()
    te = test_full_df.copy()

    if "cons_ppp17" not in tr.columns:
        raise ValueError("train_full_df에 cons_ppp17이 필요합니다.")
    if "survey_id" not in tr.columns or "survey_id" not in te.columns:
        raise ValueError("train/test에 survey_id가 필요합니다.")
    if "weight" not in tr.columns or "weight" not in te.columns:
        raise ValueError("train/test에 weight가 필요합니다.")

    # log target
    tr = tr[tr["cons_ppp17"].notna() & (tr["cons_ppp17"] > 0)].copy()
    tr["log_cons"] = np.log(tr["cons_ppp17"].astype(float))

    groups = tr["survey_id"].astype(str).values
    y_raw_log = tr["log_cons"].values

    # params default
    if lgb_params is None:
        lgb_params = dict(
            n_estimators=8000,
            learning_rate=0.011037005084629353,
            num_leaves=32,
            min_child_samples=80,
            subsample=0.9943172425614316,
            colsample_bytree=0.6340596394027656,
            reg_lambda=0.017229667246557494,
            random_state=seed,
            n_jobs=-1
        )
    if xgb_params is None:
        xgb_params = dict(
            n_estimators=4500,
            learning_rate=0.01374925873088615,
            max_depth=5,
            subsample=0.7299796125452439,
            colsample_bytree=0.6312082363473505,
            reg_lambda=0.002584212748040726,
            min_child_weight=9.320918627855733,
            gamma=2.5538369849854422e-05,
            objective="reg:squarederror",
            random_state=seed,
            n_jobs=-1
        )
    if cat_params is None:
        cat_params = dict(
            iterations=7500,
            learning_rate=0.013781131984802358,
            depth=9,
            l2_leaf_reg=0.007607411541415501,
            random_seed=seed,
            loss_function="RMSE",
            verbose=False
        )

    n_tr = len(tr)
    oof_logmu = np.zeros(n_tr, dtype=float)
    y_oof_clip = np.zeros(n_tr, dtype=float)

    # test 예측은 fold별 meta 예측을 평균
    test_logmu_folds = []

    gkf = GroupKFold(n_splits=n_splits)

    for fold, (tr_idx, va_idx) in enumerate(gkf.split(tr, y_raw_log, groups=groups), 1):
        tr_fold = tr.iloc[tr_idx].copy()
        va_fold = tr.iloc[va_idx].copy()

        # fold별 y clip (train 기준)
        lo = float(np.quantile(tr_fold["log_cons"].values, q_low))
        hi = float(np.quantile(tr_fold["log_cons"].values, q_high))
        y_tr = tr_fold["log_cons"].clip(lo, hi).values
        y_va = va_fold["log_cons"].clip(lo, hi).values
        y_oof_clip[va_idx] = y_va

        # fold별 전처리 fit/transform
        X_tr, st = prepare_features(tr_fold, state=None, fit=True)
        X_va, _  = prepare_features(va_fold, state=st, fit=False)
        X_te, _  = prepare_features(te, state=st, fit=False)

        # 컬럼명 정리 + train 기준 정렬
        orig_cols = list(X_tr.columns)
        new_cols = clean_cols_unique(orig_cols)
        rename_map = dict(zip(orig_cols, new_cols))

        X_tr = X_tr.rename(columns=rename_map)
        X_va = X_va.rename(columns=rename_map).reindex(columns=X_tr.columns, fill_value=0)
        X_te = X_te.rename(columns=rename_map).reindex(columns=X_tr.columns, fill_value=0)

        # sample_weight (가능하면 사용, sqrt로 완화)
        w_tr = tr_fold["weight"].astype(float).values
        w_va = va_fold["weight"].astype(float).values
        w_tr_use = np.sqrt(np.maximum(w_tr, 0))
        w_va_use = np.sqrt(np.maximum(w_va, 0))

        # base models
        m_lgb = lgb.LGBMRegressor(**lgb_params)
        try:
            m_lgb.fit(
                X_tr, y_tr,
                sample_weight=w_tr_use,
                eval_set=[(X_va, y_va)],
                eval_sample_weight=[w_va_use],
                eval_metric="rmse",
                callbacks=[lgb.early_stopping(lgb_early_stopping, verbose=False)]
            )
        except TypeError:
            m_lgb.fit(
                X_tr, y_tr,
                sample_weight=w_tr_use,
                eval_set=[(X_va, y_va)],
                eval_metric="rmse",
                callbacks=[lgb.early_stopping(lgb_early_stopping, verbose=False)]
            )
        pred_lgb_va = m_lgb.predict(X_va, num_iteration=getattr(m_lgb, "best_iteration_", None))
        pred_lgb_te = m_lgb.predict(X_te, num_iteration=getattr(m_lgb, "best_iteration_", None))

        m_xgb = xgb.XGBRegressor(**xgb_params)
        try:
            m_xgb.fit(
                X_tr, y_tr,
                sample_weight=w_tr_use,
                eval_set=[(X_va, y_va)],
                sample_weight_eval_set=[w_va_use],
                # eval_metric="rmse",  # Removed this line
                # early_stopping_rounds=xgb_early_stopping, # Removed this line
                verbose=False
            )
        except TypeError:
            m_xgb.fit(
                X_tr, y_tr,
                sample_weight=w_tr_use,
                eval_set=[(X_va, y_va)],
                # eval_metric="rmse",  # Removed this line
                # early_stopping_rounds=xgb_early_stopping, # Removed this line
                verbose=False
            )
        best_it = getattr(m_xgb, "best_iteration", None)
        if best_it is None:
            pred_xgb_va = m_xgb.predict(X_va)
            pred_xgb_te = m_xgb.predict(X_te)
        else:
            pred_xgb_va = m_xgb.predict(X_va, iteration_range=(0, int(best_it) + 1))
            pred_xgb_te = m_xgb.predict(X_te, iteration_range=(0, int(best_it) + 1))

        cat_params_fold = dict(cat_params)
        cat_params_fold.update(dict(use_best_model=True, od_type="Iter", od_wait=int(cat_od_wait)))
        m_cat = CatBoostRegressor(**cat_params_fold)
        m_cat.fit(
            X_tr, y_tr,
            sample_weight=w_tr_use,
            eval_set=(X_va, y_va),
            verbose=False
        )
        pred_cat_va = m_cat.predict(X_va)
        pred_cat_te = m_cat.predict(X_te)

        # meta train/val/test (fold train으로만 fit)
        P_tr = np.vstack([
            m_lgb.predict(X_tr, num_iteration=getattr(m_lgb, "best_iteration_", None)),
            (m_xgb.predict(X_tr, iteration_range=(0, int(best_it) + 1)) if best_it is not None else m_xgb.predict(X_tr)),
            m_cat.predict(X_tr)
        ]).T

        P_va = np.vstack([pred_lgb_va, pred_xgb_va, pred_cat_va]).T
        P_te = np.vstack([pred_lgb_te, pred_xgb_te, pred_cat_te]).T

        meta = Pipeline(steps=[
            ("scaler", StandardScaler(with_mean=True, with_std=True)),
            ("enet", ElasticNetCV(
                l1_ratio=[0.05, 0.1, 0.2, 0.4, 0.6, 0.8, 0.95],
                alphas=np.logspace(-4, 1, 40),
                cv=int(meta_cv),
                fit_intercept=True,
                positive=bool(positive_meta),
                random_state=seed,
                max_iter=20000
            ))
        ])
        meta.fit(P_tr, y_tr)

        pred_meta_va = meta.predict(P_va)
        pred_meta_te = meta.predict(P_te)

        oof_logmu[va_idx] = pred_meta_va
        test_logmu_folds.append(pred_meta_te)

        rmse_fold = float(np.sqrt(mean_squared_error(y_va, pred_meta_va)))
        print(f"[Fold {fold}] meta log RMSE = {rmse_fold:.5f}")

    # test는 fold 평균
    test_logmu = np.mean(np.vstack(test_logmu_folds), axis=0)

    # base_rmse_log (alpha 튜닝 기준)
    base_rmse_log = float(np.sqrt(np.mean((y_oof_clip - oof_logmu) ** 2)))

    return oof_logmu, test_logmu, y_oof_clip, base_rmse_log


# ============================================================
# 4) 통합 실행: Stacking OOF/Test -> Alpha 튜닝(CDF) -> 최종 rate 예측
# ============================================================

def pipeline_sigma_model_log_cdf_with_stacking(
    train_full_df,
    test_full_df,
    train_rates_gt,
    thresholds,
    survey_col="survey_id",
    weight_col="weight",
    n_splits=3,
    seed=42,
    lgb_params=None,
    xgb_params=None,
    cat_params=None
):
    # 1) stacking
    oof_logmu, test_logmu, y_oof_clip, _ = get_fold_logmu_preds_stacking(
        train_full_df=train_full_df,
        test_full_df=test_full_df,
        n_splits=n_splits,
        seed=seed,
        lgb_params=lgb_params,
        xgb_params=xgb_params,
        cat_params=cat_params
    )

    # 2) sigma GT (train only)
    train_eval = train_full_df.loc[
        train_full_df["cons_ppp17"].notna() & (train_full_df["cons_ppp17"] > 0),
        [survey_col, weight_col]
    ].copy()
    train_eval["oof_logmu"] = oof_logmu
    train_eval["log_cons"] = np.log(train_full_df.loc[train_eval.index, "cons_ppp17"])

    sigma_gt = compute_survey_sigma_gt(
        train_eval,
        mu_col="oof_logmu",
        true_log_col="log_cons",
        survey_col=survey_col,
        weight_col=weight_col
    )

    # 3) survey-level features
    feat_train = make_survey_features_from_logmu(
        train_eval, "oof_logmu", survey_col
    )
    feat_train = feat_train.merge(sigma_gt, on=survey_col, how="left")

    X_sig = feat_train.drop(columns=[survey_col, "sigma_gt"])
    y_sig = feat_train["sigma_gt"]

    sigma_model = CatBoostRegressor(
        iterations=2000,
        depth=6,
        learning_rate=0.03,
        loss_function="RMSE",
        random_seed=seed,
        verbose=False
    )
    sigma_model.fit(X_sig, y_sig)

    # 4) test sigma prediction
    test_eval = test_full_df[[survey_col]].copy()
    test_eval["pred_logmu"] = test_logmu

    feat_test = make_survey_features_from_logmu(
        test_eval, "pred_logmu", survey_col
    )
    feat_test["sigma_hat"] = sigma_model.predict(
        feat_test.drop(columns=[survey_col])
    )

    sigma_map = dict(zip(feat_test[survey_col], feat_test["sigma_hat"]))
    test_eval["sigma_hat"] = test_eval[survey_col].map(sigma_map)

    # 5) final rate
    test_eval[weight_col] = test_full_df[weight_col].values

    final_rates = compute_survey_rates_CDF_log_sigma(
        hh_df=test_eval,
        mu_log_col="pred_logmu",
        sigma_col="sigma_hat",
        survey_col=survey_col,
        weight_col=weight_col,
        thresholds=thresholds,
        monotone_fix=True
    )

    return final_rates, test_logmu


# ============================================================
# 5) 실행 예시 (너가 이미 갖고 있는 변수 기준)
# ============================================================

# thresholds 추출
thresholds, ordered_rate_cols, rates_survey_col = infer_thresholds_from_train_rates(train_rates_gt)
print("Thresholds detected:", len(thresholds))

# train_rates_gt의 survey_col 정리 (필요 시)
train_rates_gt_fixed = train_rates_gt.rename(columns={rates_survey_col: "survey_id"}).copy()

# Define train_full and test_full
train_full = df
test_full = test_hh_features

# optional: base model params (원하면 너 튜닝값 넣어도 됨)
lgb_params = None
xgb_params = None
cat_params = None

# Modified the function call to capture test_logmu
final_rates_df_result, test_logmu_result = pipeline_sigma_model_log_cdf_with_stacking(
    train_full_df=df,
    test_full_df=test_hh_features,
    train_rates_gt=train_rates_gt_fixed,
    thresholds=thresholds,
    survey_col="survey_id",
    weight_col="weight"
)

rename_map = {
    f"pct_hh_below_{thr:.2f}": col
    for thr, col in zip(thresholds, ordered_rate_cols)
}
print(final_rates_df_result.head())
submission = final_rates_df_result.rename(columns=rename_map)
submission.to_csv("submission_sigma_model.csv", index=False)
print("Saved submission_sigma_model.csv")

# Assign to global variables that the next cell expects
final_rates_df = final_rates_df_result
test_logmu = test_logmu_result

Thresholds detected: 19
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.047004 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2002
[LightGBM] [Info] Number of data points in the train set: 66772, number of used features: 128
[LightGBM] [Info] Start training from score 2.160551
[Fold 1] meta log RMSE = 0.33378
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.058068 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1987
[LightGBM] [Info] Number of data points in the train set: 69650, number of used features: 129
[LightGBM] [Info] Start training from score 2.169405
[Fold 2] meta log RMSE = 0.33483
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.086141 s

/tmp/ipython-input-1703902397.py:98: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: np.sqrt(np.average(g.resid2, weights=g[weight_col])))


   survey_id  pct_hh_below_3.17  pct_hh_below_3.94  pct_hh_below_4.60  \
0     400000           0.050216           0.097544           0.145102   
1     500000           0.043728           0.089654           0.137254   
2     600000           0.044964           0.089282           0.134852   

   pct_hh_below_5.26  pct_hh_below_5.88  pct_hh_below_6.47  pct_hh_below_7.06  \
0           0.196088           0.245289           0.292141           0.338229   
1           0.189095           0.239544           0.287778           0.335299   
2           0.184701           0.233631           0.280838           0.327732   

   pct_hh_below_7.70  pct_hh_below_8.40  pct_hh_below_9.13  pct_hh_below_9.87  \
0           0.386684           0.437241           0.486776           0.533388   
1           0.385262           0.437337           0.488259           0.536054   
2           0.377400           0.429502           0.480721           0.528985   

   pct_hh_below_10.70  pct_hh_below_11.62  pct_hh_below_1

In [ ]:
# ============================================================
# Submission 만들기 (submission.zip)
# - root level에 2개 CSV 포함:
#   1) predicted_household_consumption.csv
#   2) predicted_poverty_distribution.csv
# ============================================================

import os
import zipfile
import numpy as np
import pandas as pd

# -------------------------------
# [전제]
# 아래 변수들이 이미 만들어져 있어야 함:
# 1) test_full : test household 단위 DataFrame (survey_id, hhid(or household_id) 포함)
# 2) (stacking+CDF) 파이프라인 결과:
#    - test_logmu : test household별 log(consumption) 예측 (길이 = len(test_full))
#    - final_rates_df : survey별 poverty distribution DataFrame
#      (컬럼: survey_id, pct_hh_below_3.17 ... pct_hh_below_27.37)
#
# 만약 너가 위에서 만든 함수 리턴을 그대로 받았다면,
# final_rates_df 는 `final_rates_df`로 있고,
# test_logmu는 pipeline에서 따로 리턴하지 않으니 아래에서 다시 만들거나,
# pipeline 함수에서 test_logmu를 추가로 리턴하도록 수정해야 함.
# -------------------------------


# ============================================================
# 1) predicted_household_consumption.csv 만들기
#    - 요구: survey_id, household_id, per_capita_household_consumption
# ============================================================

# (A) test_logmu 준비
# 만약 네 코드에서 test_logmu를 이미 갖고 있으면 그대로 사용.
# 없으면 아래처럼 pipeline에서 나온 test_logmu를 변수로 확보해야 함.

# 여기서는 test_logmu가 이미 존재한다고 가정:
# test_logmu: shape (len(test_full), )

if "test_logmu" not in globals():
    raise ValueError(
        "test_logmu 변수가 없습니다. "
        "stacking 예측 함수(get_fold_logmu_preds_stacking)에서 test_logmu를 받아서 저장해두세요."
    )

# (B) log -> raw consumption (per capita)
test_pred_cons = np.exp(np.asarray(test_logmu, dtype=float))

# (C) household consumption submission DF
hh_sub = pd.DataFrame({
    "survey_id": test_full["survey_id"].astype(int),
    "hhid": test_full["hhid"].astype(int),
    "cons_ppp17": test_pred_cons.astype(float)
})

# (D) 정렬(권장: survey_id, household_id)
hh_sub = hh_sub.sort_values(["survey_id", "hhid"]).reset_index(drop=True)

# (E) 행 수/결측 체크(필수 안정장치)
if hh_sub.isna().any().any():
    bad = hh_sub.isna().sum()
    raise ValueError(f"household submission에 NaN이 있습니다:\n{bad}")

if len(hh_sub) != len(test_full):
    raise ValueError(f"행 수 불일치: household submission({len(hh_sub)}) != test_full({len(test_full)})")


# ============================================================
# 2) predicted_poverty_distribution.csv 만들기
#    - 요구: survey_id + 19개 threshold 컬럼
#    - final_rates_df는 이미 (survey_id, pct_hh_below_3.17 ... ) 형태여야 함
# ============================================================

if "final_rates_df" not in globals():
    raise ValueError(
        "final_rates_df 변수가 없습니다. "
        "stacking+CDF 파이프라인 결과(final_rates_df)를 받아서 저장해두세요."
    )

rate_sub = final_rates_df.copy()

# (A) survey_id 타입/정렬
rate_sub["survey_id"] = rate_sub["survey_id"].astype(int)
rate_sub = rate_sub.sort_values("survey_id").reset_index(drop=True)

# (B) 필요한 컬럼 개수 체크: 1(survey_id)+19(threshold) = 20
expected_rate_cols = ["survey_id"] + [c for c in rate_sub.columns if c.startswith("pct_hh_below_")]
if len(expected_rate_cols) != 20:
    raise ValueError(
        f"poverty distribution 컬럼 개수가 요구사항과 다릅니다. "
        f"(현재 {len(expected_rate_cols)}개; 요구 20개)\n"
        f"현재 threshold 컬럼들: {[c for c in rate_sub.columns if c.startswith('pct_hh_below_')]}"
    )

# (C) threshold 컬럼들을 숫자 기준으로 정렬(3.17 -> 27.37 순서)
thr_cols = [c for c in rate_sub.columns if c.startswith("pct_hh_below_")]
thr_pairs = []
for c in thr_cols:
    # c = "pct_hh_below_3.17"
    m = re.search(r"(\d+(\.\d+)?)", c)
    if m:
        thr_pairs.append((float(m.group(1)), c))
thr_pairs.sort(key=lambda x: x[0])
ordered_thr_cols = [c for _, c in thr_pairs]

rate_sub = rate_sub[["survey_id"] + ordered_thr_cols].copy()

# (D) 값 범위 체크: [0,1]
for c in ordered_thr_cols:
    if ((rate_sub[c] < -1e-9) | (rate_sub[c] > 1 + 1e-9)).any():
        raise ValueError(f"rate 컬럼 {c}에 [0,1] 범위를 벗어난 값이 있습니다.")

# (E) survey 개수 체크(문서상 3개여야 함)
# 대회 데이터가 3개 survey가 맞다는 전제일 때만 강제
# 아니라면 아래 if 블록은 주석 처리
if len(rate_sub) != 3:
    print(f"[경고] poverty distribution 행 수가 3이 아닙니다: {len(rate_sub)}행 (데이터 스펙 확인 필요)")

# ============================================================
# 3) 파일 저장 + ZIP 생성 (root level 2 files)
# ============================================================

out_dir = "submission_build"
os.makedirs(out_dir, exist_ok=True)

hh_path = os.path.join(out_dir, "predicted_household_consumption.csv")
rate_path = os.path.join(out_dir, "predicted_poverty_distribution.csv")
zip_path = os.path.join(out_dir, "submission.zip")

hh_sub.to_csv(hh_path, index=False)
rate_sub.to_csv(rate_path, index=False)

# ZIP: root level에 두 파일만 들어가도록 arcname 지정
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    zf.write(hh_path, arcname="predicted_household_consumption.csv")
    zf.write(rate_path, arcname="predicted_poverty_distribution.csv")

print("✅ Saved:")
print(" -", hh_path)
print(" -", rate_path)
print("✅ Zip created:")
print(" -", zip_path)

✅ Saved:
 - submission_build/predicted_household_consumption.csv
 - submission_build/predicted_poverty_distribution.csv
✅ Zip created:
 - submission_build/submission.zip
